In [ ]:
library(loadeR) 
library(loadeR.2nc) 
library(transformeR)
library(downscaleR) 
library(visualizeR)
library(downscaleR.keras) 
library(climate4R.value) 
library(climate4R.UDG) 
library(magrittr) 
library(RColorBrewer)
library(gridExtra)
library(ggplot2)

In [ ]:
z500<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_g500_2000-2020_era5.nc",var = "z",years = 2001:2020)
new_z500<-gridArithmetics(z500, 9.80665, operator = "/")
z700<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_g700_2000-2020_era5.nc",var = "z",years = 2001:2020)
new_z700<-gridArithmetics(z700, 9.80665, operator = "/")
z850<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_g850_2000-2020_era5.nc",var = "z",years = 2001:2020)
new_z850<-gridArithmetics(z850, 9.80665, operator = "/")

In [ ]:
q500<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_hus500_2000-2020_era5.nc",var = "q",years = 2001:2020)
q700<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_hus700_2000-2020_era5.nc",var = "q",years = 2001:2020)
q850<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_hus850_2000-2020_era5.nc",var = "q",years = 2001:2020)

In [ ]:
t500<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_ta500_2000-2020_era5.nc",var = "t",years = 2001:2020)
t700<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_ta700_2000-2020_era5.nc",var = "t",years = 2001:2020)
t850<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_ta850_2000-2020_era5.nc",var = "t",years = 2001:2020)

In [ ]:
u500<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_u500_2000-2020_era5.nc",var = "u",years = 2001:2020)
u700<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_u700_2000-2020_era5.nc",var = "u",years = 2001:2020)
u850<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_u850_2000-2020_era5.nc",var = "u",years = 2001:2020)

In [ ]:
v500<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_v500_2000-2020_era5.nc",var = "v",years = 2001:2020)
v700<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_v700_2000-2020_era5.nc",var = "v",years = 2001:2020)
v850<-loadGridData(dataset = "sensitive_experiment/8degree_20000601-20201231/8degree_v850_2000-2020_era5.nc",var = "v",years = 2001:2020)

In [ ]:
x<-makeMultiGrid(new_z500,new_z700,new_z850,q500,q700,q850,
                t500,t700,t850,u500,u700,u850,v500,v700,v850) %>% redim(,drop=T)

In [ ]:
y<-loadGridData(dataset = "era/2001-2020_prec.nc",var = "prec")

In [ ]:
xT<-subsetGrid(x,years=2001:2015) 
yT<-subsetGrid(y,years=2001:2015)
xt<-subsetGrid(x,years=2016:2020)
yt<-subsetGrid(y,years=2016:2020)

In [ ]:
xt<-scaleGrid(xt,xT,type="standardize",spatial.frame="gridbox")%>%redim(drop=TRUE)
xT<-scaleGrid(xT,type="standardize",spatial.frame="gridbox")%>%redim(drop=TRUE)

In [ ]:
xy.T <- prepareData.keras(xT,binaryGrid(gridArithmetics(yT,0.99,operator = "-"),
                                        condition = "GE",
                                        threshold = 0,
                                        partial = TRUE),
                          first.connection = "conv",
                          last.connection = "dense",
                          channels = "last")

In [ ]:
xy.tT <- prepareNewData.keras(xT,xy.T)
xy.t <- prepareNewData.keras(xt,xy.T)

In [ ]:
modelCNN <- function(inp) {
    inputs <- layer_input(shape = dim(inp$x.global)[2:4])
    l1 = layer_conv_2d(inputs,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l3 = layer_conv_2d(l2,filters = 1, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l4 = layer_flatten(l3)
    l51 = layer_dense(l4,units = dim(inp$y$Data)[2], activation = 'sigmoid') 
    l52 = layer_dense(l4,units = dim(inp$y$Data)[2], activation = 'linear') 
    l53 = layer_dense(l4,units = dim(inp$y$Data)[2], activation = 'linear') 
    outputs <- layer_concatenate(list(l51,l52,l53),trainable=TRUE)      
    model <- keras_model(inputs = inputs, outputs = outputs) 
}

In [ ]:
downscaleTrain.keras(obj = xy.T,model = modelCNN(xy.T),clear.session = TRUE,
                     compile.args = list("loss" = bernouilliGammaLoss(last.connection = "dense"),
                                         "optimizer" = optimizer_adam(lr = 0.0001)),
                     fit.args = list("batch_size" = 100,"epochs" = 10000,"validation_split" = 0.1,
                                     "verbose" = 1,
                                     "callbacks" = list(callback_early_stopping(patience = 30),
                                                        callback_model_checkpoint(filepath='./models/Sensitivity_experiment8/CNN_Sensitivity8.h5',
                                                                                  monitor='val_loss', save_best_only=TRUE))))

In [ ]:
pred_ocu_train <- downscalePredict.keras(newdata = xy.tT,C4R.template = yT,clear.session = TRUE,loss = "bernouilliGammaLoss",
                                         model = list("filepath" = './models/Sensitivity_experiment8/CNN_Sensitivity8.h5',"custom_objects" = c("custom_loss" = bernouilliGammaLoss(last.connection = "dense")))) %>% subsetGrid(var = "p")  


In [ ]:
pred <- downscalePredict.keras(newdata = xy.t,C4R.template = yt,clear.session = TRUE,loss = "bernouilliGammaLoss",
                                   model = list("filepath" = './models/Sensitivity_experiment8/CNN_Sensitivity8.h5',
                                                "custom_objects" = c("custom_loss" = bernouilliGammaLoss(last.connection = "dense"))))

In [ ]:
pred2 <- computeRainfall(log_alpha = subsetGrid(pred,var = "log_alpha"),log_beta = subsetGrid(pred,var = "log_beta"),bias = 1,simulate = TRUE)

In [ ]:
pred3<- pred2 %>% gridArithmetics(binaryGrid(subsetGrid(pred,var = "p"),ref.obs = binaryGrid(yT,threshold = 1, condition = "GE"),ref.pred = pred_ocu_train))  

In [ ]:
pred3$Dates<-yt$Dates

In [ ]:
grid2nc(pred3,NetCDFOutFile = "./Data/Sensitivity_experiment8/CNN_Precipitation_Sensitivity8.nc")